In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import Adam

from sklearn import preprocessing

Using TensorFlow backend.
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type

## データの読み込み，整形

In [3]:
# 生データの読み込み
data = pd.read_csv('./dataset/log-2020-02-20-13-54-32.csv',
                   index_col=0)
print(data.shape)
data.head()

(2442, 4)


,acceleration_x,acceleration_y,acceleration_z,time
0,-108.0,119.0,870.0,0.000000
1,-106.0,110.0,865.0,0.176580
2,-109.0,125.0,863.0,0.345388
3,-111.0,118.0,863.0,0.499721
4,-110.0,116.0,862.0,0.655409


In [4]:
# 各状態のデータに切り分け(timeは除外)
sitting = data.iloc[:110, :3]
walking = data.iloc[120:240, :3]
running = data.iloc[240:360, :3]
behaviors = {'sitting': sitting,
             'walking': walking,
             'running': running}

In [8]:
# ラベル付けして一つのデータにまとめる
for k, v in behaviors.items():
    v['class'] = k
merged_data = pd.concat([sitting, walking, running])

# 現状だと立ち上がり時[110:120]を除外しているのでindexを振り直す
merged_data = merged_data.reset_index(drop=True)

print(merged_data.shape)
merged_data.head()

(350, 4)


,acceleration_x,acceleration_y,acceleration_z,class
0,-108.0,119.0,870.0,sitting
1,-106.0,110.0,865.0,sitting
2,-109.0,125.0,863.0,sitting
3,-111.0,118.0,863.0,sitting
4,-110.0,116.0,862.0,sitting


## 前処理

In [24]:
sensor_data = merged_data.iloc[:, :3]  # classは除外
row, col = sensor_data.shape
X = np.zeros((row, col))
for i in range(row):
    X[i] = list(sensor_data.loc[i])

# 特徴量の標準化
ss = preprocessing.StandardScaler()
X_standardization = ss.fit_transform(X)
X_standardization[:5]

array([[1.06086813, 1.19895989, 1.97787953],
       [1.21245165, 0.68741666, 0.99399972],
       [0.98507637, 1.53998871, 0.60044779],
       [0.83349284, 1.14212175, 0.60044779],
       [0.9092846 , 1.02844548, 0.40367183]])

In [23]:
label = merged_data['class']
y = pd.get_dummies(label)  # one-hot vectorに変換
y.head()

,running,sitting,walking
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0


## モデルの構築

In [ ]:
batch_size = 16
num_classes = 3
epochs = 10

model = Sequential()
